# This markdown file records the training of models that did not perform as well in predicting D0-D4 values

#### Train MultiOutputRegressor(RandomForestRegressor())

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

# Load and prepare data
# df = pd.read_csv("final_historical.csv")
df = final_historical_df
df = df.dropna()

# Convert 'date' column to datetime if not already
df['date'] = pd.to_datetime(df['date'])

input_cols = ['airtemp', 'baseflow', 'ev', 'rainfall', 'runoff', 'sm', 'snowfall', 'snowwater']
target_cols = ['D0', 'D1', 'D2', 'D3', 'D4']

# Step 1: Simplify Lagged Features (Keep only 1 lag feature for each input column)
for col in input_cols:
    df[f'{col}_lag1'] = df[col].shift(1)  # 1-month lag

# Drop rows with missing values created by lagging
df = df.dropna()

# Step 2: Create Time-Aware Train/Test Split (based on date)
X_train = df[df['date'] < '2000-01-01'][input_cols + [f'{col}_lag1' for col in input_cols]]
y_train = df[df['date'] < '2000-01-01'][target_cols]
X_test = df[df['date'] >= '2000-01-01'][input_cols + [f'{col}_lag1' for col in input_cols]]
y_test = df[df['date'] >= '2000-01-01'][target_cols]

# Step 3: Scale the Features (Input columns + lagged features)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 1️⃣ RANDOM FOREST MODEL
rf_model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, max_depth=6, random_state=42))
rf_model.fit(X_train_scaled, y_train)
rf_pred = rf_model.predict(X_test_scaled)

# Save model
joblib.dump(rf_model, "random_forest_drought.pkl")

# Evaluation
rf_mse = mean_squared_error(y_test, rf_pred, multioutput='raw_values')
rf_r2 = r2_score(y_test, rf_pred, multioutput='raw_values')
rf_accuracy = 1 - np.mean(rf_mse / np.var(y_test, axis=0))

print("🌲 Random Forest Performance:")
for i, col in enumerate(target_cols):
    print(f"{col}: MSE = {rf_mse[i]:.4f}, R² = {rf_r2[i]:.4f}")
print(f"Overall pseudo-accuracy (1 - normalized MSE): {rf_accuracy:.4f}\n")


# 2️⃣ XGBOOST MODEL
xgb_model = MultiOutputRegressor(XGBRegressor(n_estimators=100, learning_rate=0.05, max_depth=4, verbosity=0))
xgb_model.fit(X_train_scaled, y_train)
xgb_pred = xgb_model.predict(X_test_scaled)

# Save model
joblib.dump(xgb_model, "xgboost_drought.pkl")

# Evaluation
xgb_mse = mean_squared_error(y_test, xgb_pred, multioutput='raw_values')
xgb_r2 = r2_score(y_test, xgb_pred, multioutput='raw_values')
xgb_accuracy = 1 - np.mean(xgb_mse / np.var(y_test, axis=0))

print("⚡ XGBoost Performance:")
for i, col in enumerate(target_cols):
    print(f"{col}: MSE = {xgb_mse[i]:.4f}, R² = {xgb_r2[i]:.4f}")
print(f"Overall pseudo-accuracy (1 - normalized MSE): {xgb_accuracy:.4f}\n")


# 🔍 Visualization (XGBoost)
for i, col in enumerate(target_cols):
    plt.figure(figsize=(6, 4))
    plt.scatter(y_test[col], xgb_pred[:, i], alpha=0.5, edgecolor='k')
    plt.plot([y_test[col].min(), y_test[col].max()],
             [y_test[col].min(), y_test[col].max()],
             'r--', lw=2)
    plt.title(f"{col} Prediction (XGBoost)")
    plt.xlabel("Actual")
    plt.ylabel("Predicted")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

## LSTM Training

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# --- LOAD & PREP DATA ---
# df = pd.read_csv("final_historical.csv")
df = final_historical_df
df['date'] = pd.to_datetime(df['date'])
df = df.dropna()

input_cols = ['airtemp', 'baseflow', 'ev', 'rainfall', 'runoff', 'sm', 'snowfall', 'snowwater']
target_cols = ['D0', 'D1', 'D2', 'D3', 'D4']

# --- SCALE FEATURES ---
scaler = StandardScaler()
df[input_cols] = scaler.fit_transform(df[input_cols])

# --- CREATE SEQUENCES ---
def create_sequences(df, input_cols, target_cols, seq_len=12):
    X, y = [], []
    for i in range(len(df) - seq_len):
        X.append(df[input_cols].iloc[i:i+seq_len].values)
        y.append(df[target_cols].iloc[i+seq_len].values)  # predict drought at next time step
    return np.array(X), np.array(y)

SEQ_LEN = 12
X_seq, y_seq = create_sequences(df, input_cols, target_cols, SEQ_LEN)

# --- TIME-AWARE SPLIT ---
split_idx = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:split_idx], X_seq[split_idx:]
y_train, y_test = y_seq[:split_idx], y_seq[split_idx:]

# --- LSTM MODEL ---
model = Sequential([
    LSTM(64, input_shape=(SEQ_LEN, len(input_cols)), return_sequences=False),
    Dense(32, activation='relu'),
    Dense(len(target_cols))  # one output per drought class
])

model.compile(optimizer='adam', loss='mse')
early_stop = EarlyStopping(patience=10, restore_best_weights=True)

# --- TRAIN ---
history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=[early_stop],
                    verbose=1)

# --- EVALUATE ---
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
r2 = r2_score(y_test, y_pred, multioutput='raw_values')
pseudo_acc = 1 - np.mean(mse / np.var(y_test, axis=0))

print("📉 LSTM Performance:")
for i, col in enumerate(target_cols):
    print(f"{col}: MSE = {mse[i]:.4f}, R² = {r2[i]:.4f}")
print(f"Overall pseudo-accuracy (1 - normalized MSE): {pseudo_acc:.4f}\n")

# --- PLOT PREDICTIONS ---
for i, col in enumerate(target_cols):
    plt.figure(figsize=(6, 4))
    plt.plot(y_test[:, i], label="Actual")
    plt.plot(y_pred[:, i], label="Predicted")
    plt.title(f"{col} Prediction (LSTM)")
    plt.xlabel("Time Steps")
    plt.ylabel("Value")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
# === IMPORTS ===
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# === LOAD & PREPROCESS ===
# df = pd.read_csv("final_historical.csv")
df = final_historical_df
df.dropna(inplace=True)
df['date'] = pd.to_datetime(df['date'])

input_cols = ['airtemp', 'baseflow', 'ev', 'rainfall', 'runoff', 'sm', 'snowfall', 'snowwater']
target_cols = ['D0', 'D1', 'D2', 'D3', 'D4']
SEQ_LEN = 3  # how many months of memory

# === SCALING ===
scaler_X = StandardScaler()
scaler_y = StandardScaler()
df[input_cols] = scaler_X.fit_transform(df[input_cols])
df[target_cols] = scaler_y.fit_transform(df[target_cols])

# === CREATE SEQUENCES ===
def create_sequences(df, input_cols, target_cols, seq_len):
    X, y = [], []
    for i in range(len(df) - seq_len):
        X.append(df[input_cols].iloc[i:i+seq_len].values)
        y.append(df[target_cols].iloc[i+seq_len].values)
    return np.array(X), np.array(y)

X_seq, y_seq = create_sequences(df, input_cols, target_cols, SEQ_LEN)

# === TIME-AWARE SPLIT ===
dates = df['date'].iloc[SEQ_LEN:].reset_index(drop=True)
split_date = pd.Timestamp('2000-01-01')
train_idx = dates < split_date
test_idx = dates >= split_date

X_train, y_train = X_seq[train_idx], y_seq[train_idx]
X_test, y_test = X_seq[test_idx], y_seq[test_idx]

# === MODEL BUILDING ===
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(SEQ_LEN, len(input_cols))),
    Dropout(0.2),
    LSTM(64),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(len(target_cols))
])
model.compile(optimizer=Adam(learning_rate=0.001), loss='mae')

early_stop = EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)

# === TRAINING ===
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# === PREDICTION & INVERSE SCALING ===
y_pred = model.predict(X_test)
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test)

# === EVALUATION ===
mse = mean_squared_error(y_test_inv, y_pred_inv, multioutput='raw_values')
r2 = r2_score(y_test_inv, y_pred_inv, multioutput='raw_values')
pseudo_acc = 1 - np.mean(mse / np.var(y_test_inv, axis=0))

print("📉 LSTM Performance:")
for i, col in enumerate(target_cols):
    print(f"{col}: MSE = {mse[i]:.4f}, R² = {r2[i]:.4f}")
print(f"Overall pseudo-accuracy (1 - normalized MSE): {pseudo_acc:.4f}")

# === PLOT RESULTS ===
for i, col in enumerate(target_cols):
    plt.figure(figsize=(6, 4))
    plt.scatter(y_test_inv[:, i], y_pred_inv[:, i], alpha=0.5, edgecolor='k')
    plt.plot([y_test_inv[:, i].min(), y_test_inv[:, i].max()],
             [y_test_inv[:, i].min(), y_test_inv[:, i].max()],
             'r--', lw=2)
    plt.title(f"{col} Prediction (LSTM)")
    plt.xlabel("Actual")
    plt.ylabel("Predicted")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Dropout, Attention, LayerNormalization, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# === LOAD & CLEAN ===
# df = pd.read_csv("final_historical.csv")
df = final_historical_df
df.dropna(inplace=True)
df['date'] = pd.to_datetime(df['date'])

input_cols = ['airtemp', 'baseflow', 'ev', 'rainfall', 'runoff', 'sm', 'snowfall', 'snowwater']
target_cols = ['D0', 'D1', 'D2', 'D3', 'D4']
SEQ_LEN = 6          # number of months to look back
PRED_HORIZON = 3     # number of future steps to predict

# === SCALE ===
scaler_X = StandardScaler()
scaler_y = StandardScaler()
df[input_cols] = scaler_X.fit_transform(df[input_cols])
df[target_cols] = scaler_y.fit_transform(df[target_cols])

# === SEQUENCE CREATION ===
def create_multi_horizon_sequences(df, input_cols, target_cols, seq_len, pred_horizon):
    X, y = [], []
    for i in range(len(df) - seq_len - pred_horizon + 1):
        X.append(df[input_cols].iloc[i:i+seq_len].values)
        y.append(df[target_cols].iloc[i+seq_len:i+seq_len+pred_horizon].values)
    return np.array(X), np.array(y)

X_seq, y_seq = create_multi_horizon_sequences(df, input_cols, target_cols, SEQ_LEN, PRED_HORIZON)

# === TIME-AWARE SPLIT ===
dates = df['date'].iloc[SEQ_LEN:SEQ_LEN + len(y_seq)].reset_index(drop=True)
split_date = pd.Timestamp("2000-01-01")
train_idx = dates < split_date
test_idx = dates >= split_date

X_train, y_train = X_seq[train_idx], y_seq[train_idx]
X_test, y_test = X_seq[test_idx], y_seq[test_idx]

# === MODEL WITH ATTENTION ===
input_layer = Input(shape=(SEQ_LEN, len(input_cols)))
x = GRU(128, return_sequences=True)(input_layer)
x = Dropout(0.2)(x)
x = LayerNormalization()(x)

# Attention block
query = Dense(64)(x)
key = Dense(64)(x)
value = Dense(64)(x)
attn_out = Attention(use_scale=True)([query, value, key])
attn_out = LayerNormalization()(attn_out)

x = Concatenate()([x, attn_out])
x = GRU(64)(x)
x = Dropout(0.2)(x)
output = Dense(PRED_HORIZON * len(target_cols))(x)
output = tf.keras.layers.Reshape((PRED_HORIZON, len(target_cols)))(output)

model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer=Adam(0.001), loss='mae')

# === TRAIN ===
early_stop = EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(patience=5, factor=0.5, verbose=1)

history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# === PREDICT & EVALUATE ===
y_pred = model.predict(X_test)
y_pred_inv = scaler_y.inverse_transform(y_pred.reshape(-1, len(target_cols))).reshape(-1, PRED_HORIZON, len(target_cols))
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, len(target_cols))).reshape(-1, PRED_HORIZON, len(target_cols))

# Evaluate at each future time step
print("📉 Multi-Horizon GRU + Attention Performance:")
for step in range(PRED_HORIZON):
    mse = mean_squared_error(y_test_inv[:, step], y_pred_inv[:, step], multioutput='raw_values')
    r2 = r2_score(y_test_inv[:, step], y_pred_inv[:, step], multioutput='raw_values')
    pseudo_acc = 1 - np.mean(mse / np.var(y_test_inv[:, step], axis=0))
    print(f"🔮 Step {step+1}:")
    for i, col in enumerate(target_cols):
        print(f"  {col}: MSE = {mse[i]:.4f}, R² = {r2[i]:.4f}")
    print(f"  Overall pseudo-accuracy: {pseudo_acc:.4f}\n")

# === PLOTS ===
for i, col in enumerate(target_cols):
    plt.figure(figsize=(6, 4))
    plt.scatter(y_test_inv[:, 0, i], y_pred_inv[:, 0, i], alpha=0.5, edgecolor='k')
    plt.plot([y_test_inv[:, 0, i].min(), y_test_inv[:, 0, i].max()],
             [y_test_inv[:, 0, i].min(), y_test_inv[:, 0, i].max()],
             'r--', lw=2)
    plt.title(f"{col} (Step 1 Forecast)")
    plt.xlabel("Actual")
    plt.ylabel("Predicted")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

### LSTM run that worked well but did not contribute to our results/conclusion

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam

# Step 1: Preprocessing and Feature Engineering
# Assuming final_historical_df is your dataframe
final_historical_df['date'] = pd.to_datetime(final_historical_df['date'])

# Create Lag Features (1-step lag for D0, D1, D2, D3, D4)
for col in ['D0', 'D1', 'D2', 'D3', 'D4']:
    for lag in range(1, 4):  # Lag 1, 2, 3
        final_historical_df[f'{col}_lag{lag}'] = final_historical_df[col].shift(lag)

# Create Rolling Mean and Std Features (using a window of 7 days, for example)
for col in ['D0', 'D1', 'D2', 'D3', 'D4']:
    final_historical_df[f'{col}_rolling_mean'] = final_historical_df[col].rolling(window=7).mean()
    final_historical_df[f'{col}_rolling_std'] = final_historical_df[col].rolling(window=7).std()

# Drop NaN values generated by shifting and rolling operations
final_historical_df = final_historical_df.dropna()

# Step 2: Prepare Features and Targets
features = final_historical_df.drop(columns=['date', 'D0', 'D1', 'D2', 'D3', 'D4'])
target = final_historical_df[['D0', 'D1', 'D2', 'D3', 'D4']]

# Normalize Features and Targets
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

X_scaled = feature_scaler.fit_transform(features)
y_scaled = target_scaler.fit_transform(target)

# Step 3: Create Sequences for LSTM
SEQ_LENGTH = 12 # 12 time steps (e.g., 12 weeks of data)
# 12 = all r^2 are above 0 (d3,4 improved)

X_seq, y_seq = [], []
for i in range(len(X_scaled) - SEQ_LENGTH):
    X_seq.append(X_scaled[i:i+SEQ_LENGTH])  # Sequence of features
    y_seq.append(y_scaled[i+SEQ_LENGTH])  # Next time step prediction

X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

# Step 4: Train/Test Split
split_idx = int(0.8 * len(X_seq))  # 80% train, 20% test
X_train, X_test = X_seq[:split_idx], X_seq[split_idx:]
y_train, y_test = y_seq[:split_idx], y_seq[split_idx:]

# Step 5: Build and Train the LSTM Model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(SEQ_LENGTH, X_train.shape[2])),
    Dropout(0.3),
    LSTM(32),
    Dense(5)  # Output size matches D0, D1, D2, D3, D4
])

learning_rate = 0.05
# Compile the model with the custom learning rate
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=['mae'])

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=200,
    batch_size=32,
    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)]
)

# Step 6: Evaluate the Model
y_pred_scaled = model.predict(X_test)

# Inverse transform predictions and true labels to original scale
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_orig = target_scaler.inverse_transform(y_test)

# Calculate MAE and R² for each drought level
from sklearn.metrics import mean_absolute_error, r2_score

for i, category in enumerate(['D0', 'D1', 'D2', 'D3', 'D4']):
    mae = mean_absolute_error(y_test_orig[:, i], y_pred[:, i])
    r2 = r2_score(y_test_orig[:, i], y_pred[:, i])
    print(f"{category} - MAE: {mae:.4f}, R²: {r2:.4f}")
    
    # Plot predictions vs true values
    plt.figure(figsize=(10, 4))
    plt.plot(y_test_orig[:, i], label='True')
    plt.plot(y_pred[:, i], label='Predicted')
    plt.title(f"{category} - True vs Predicted")
    plt.xlabel("Time Step")
    plt.ylabel("Percentage Area")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

### Training each D0-D4 label separately

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Dropout, Attention, LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# === LOAD & CLEAN ===
# df = pd.read_csv("final_historical.csv")
df = final_historical_df
df.dropna(inplace=True)
df['date'] = pd.to_datetime(df['date'])

input_cols = ['airtemp', 'baseflow', 'ev', 'rainfall', 'runoff', 'sm', 'snowfall', 'snowwater']
target_cols = ['D0', 'D1', 'D2', 'D3', 'D4']
SEQ_LEN = 6
PRED_HORIZON = 3  # multi-horizon (e.g., next 3 months)

# === SCALE INPUT FEATURES ===
scaler_X = StandardScaler()
df[input_cols] = scaler_X.fit_transform(df[input_cols])

# === CREATE SEQUENCES FOR EACH TARGET ===
def create_sequences(df, input_cols, target_col, seq_len, pred_horizon):
    X, y = [], []
    for i in range(len(df) - seq_len - pred_horizon + 1):
        X.append(df[input_cols].iloc[i:i+seq_len].values)
        y.append(df[target_col].iloc[i+seq_len:i+seq_len+pred_horizon].values)
    return np.array(X), np.array(y)

# === TIME SPLIT HELPER ===
def time_aware_split(X, y, dates, split_date):
    split_mask = dates < split_date
    return X[split_mask], y[split_mask], X[~split_mask], y[~split_mask]

# === TRAIN EACH MODEL ===
for target_col in target_cols:
    print(f"\n🚀 Training for {target_col}...\n")
    
    # Scale the target independently
    scaler_y = StandardScaler()
    df[target_col] = scaler_y.fit_transform(df[[target_col]])
    
    X_seq, y_seq = create_sequences(df, input_cols, target_col, SEQ_LEN, PRED_HORIZON)
    dates = df['date'].iloc[SEQ_LEN:SEQ_LEN + len(y_seq)].reset_index(drop=True)
    
    X_train, y_train, X_test, y_test = time_aware_split(X_seq, y_seq, dates, pd.Timestamp("2000-01-01"))
    
    # === MODEL ===
    input_layer = Input(shape=(SEQ_LEN, len(input_cols)))
    x = GRU(64, return_sequences=True)(input_layer)
    x = Dropout(0.2)(x)
    x = LayerNormalization()(x)

    # Attention layer
    query = Dense(32)(x)
    key = Dense(32)(x)
    value = Dense(32)(x)
    attn_out = Attention(use_scale=True)([query, value, key])
    x = tf.keras.layers.Concatenate()([x, attn_out])

    x = GRU(32)(x)
    x = Dropout(0.2)(x)
    output = Dense(PRED_HORIZON)(x)  # One output per month

    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer=Adam(0.001), loss='mae')

    early_stop = EarlyStopping(patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(patience=5, factor=0.5, verbose=1)

    history = model.fit(
        X_train, y_train,
        validation_split=0.1,
        epochs=100,
        batch_size=32,
        callbacks=[early_stop, reduce_lr],
        verbose=1
    )

    # Predict + Inverse Scale
    y_pred = model.predict(X_test)
    y_pred_inv = scaler_y.inverse_transform(y_pred)
    y_test_inv = scaler_y.inverse_transform(y_test)

    # === EVALUATION ===
    mse = mean_squared_error(y_test_inv, y_pred_inv, multioutput='raw_values')
    r2 = r2_score(y_test_inv, y_pred_inv, multioutput='raw_values')
    pseudo_acc = 1 - np.mean(mse / np.var(y_test_inv, axis=0))

    print(f"📉 {target_col} Performance:")
    for i in range(PRED_HORIZON):
        print(f"Step {i+1}: MSE = {mse[i]:.4f}, R² = {r2[i]:.4f}")
    print(f"Overall pseudo-accuracy: {pseudo_acc:.4f}")

    # === PLOT FIRST STEP ===
    plt.figure(figsize=(6, 4))
    plt.scatter(y_test_inv[:, 0], y_pred_inv[:, 0], alpha=0.5, edgecolor='k')
    plt.plot([y_test_inv[:, 0].min(), y_test_inv[:, 0].max()],
             [y_test_inv[:, 0].min(), y_test_inv[:, 0].max()],
             'r--', lw=2)
    plt.title(f"{target_col} Prediction (Step 1)")
    plt.xlabel("Actual")
    plt.ylabel("Predicted")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

### LSTM w/ independent drought levels as ys

In [ ]:
# Select features and target
features = final_historical_df.drop(columns=['date', 'D0', 'D1', 'D2', 'D3', 'D4'])
target = final_historical_df[['D0', 'D1', 'D2', 'D3', 'D4']]  # Multi-output regression

# Ensure proper data preparation
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

# Check if required columns exist
required_cols = ['date', 'D0', 'D1', 'D2', 'D3', 'D4']
missing_cols = [col for col in required_cols if col not in final_historical_df.columns]
if missing_cols:
    raise ValueError(f"Missing columns in final_df: {missing_cols}")

# Add temporal features
pd.to_datetime(final_historical_df['date'])

# final_historical_df['date'] = final_historical_df['date'][:-2]

# Scale features and target
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

X_scaled = feature_scaler.fit_transform(features)
y_scaled = target_scaler.fit_transform(target)

print(f"Features shape: {X_scaled.shape}, Target shape: {y_scaled.shape}")

# Create sequences for LSTM
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])  # Predict next step after sequence
    return np.array(X_seq), np.array(y_seq)

SEQ_LENGTH = 12  # Lookback period (12 weeks)
X_seq, y_seq = create_sequences(X_scaled, y_scaled, SEQ_LENGTH)

print(f"Sequence shapes - X_seq: {X_seq.shape}, y_seq: {y_seq.shape}")

# Split sequences into train and test sets
split_idx = int(0.8 * len(X_seq))  # 80% train, 20% test
X_train, X_test = X_seq[:split_idx], X_seq[split_idx:]
y_train, y_test = y_seq[:split_idx], y_seq[split_idx:]

print(f"Train shapes - X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Test shapes - X_test: {X_test.shape}, y_test: {y_test.shape}")

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define the LSTM model architecture
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(SEQ_LENGTH, X_train.shape[2])),
    Dropout(0.3),
    LSTM(32),
    Dense(5)  # Output size matches the number of drought categories (D0-D4)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)]
)

# Make predictions on test data
y_pred_scaled = model.predict(X_test)

# Inverse transform predictions and true labels to original scale
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_orig = target_scaler.inverse_transform(y_test)

print(f"Prediction shape: {y_pred.shape}, True labels shape: {y_test_orig.shape}")

# Evaluate using MAE for each drought category (D0-D4)
from sklearn.metrics import mean_absolute_error

for i, category in enumerate(['D0', 'D1', 'D2', 'D3', 'D4']):
    mae = mean_absolute_error(y_test_orig[:, i], y_pred[:, i])
    print(f"MAE for {category}: {mae:.4f}")

import matplotlib.pyplot as plt

# Plot predictions vs true values for D0 category as an example
plt.figure(figsize=(12, 6))
plt.plot(y_test_orig[:, 0], label='True D0')
plt.plot(y_pred[:, 0], label='Predicted D0')
plt.title('Predictions vs True Values for D0')
plt.xlabel('Time Steps')
plt.ylabel('Percentage Area')
plt.legend()
plt.show()

from sklearn.ensemble import RandomForestRegressor

# Flatten sequences for Random Forest (reshape to 2D)
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Train Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100)
rf_model.fit(X_train_flat, y_train)

# Predict and evaluate RF model
rf_pred = rf_model.predict(X_test_flat)
rf_mae = mean_absolute_error(y_test, rf_pred)

print(f"Random Forest MAE: {rf_mae:.4f}")

from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np

# Convert predictions and true values to categorical labels
y_test_labels = np.argmax(y_test, axis=1)
y_pred_labels = np.argmax(y_pred, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_labels, y_pred_labels)
print(f"Accuracy: {accuracy:.4f}")

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test_labels, y_pred_labels)
print("Confusion Matrix:\n", conf_matrix)

import matplotlib.pyplot as plt
import seaborn as sns

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['D0', 'D1', 'D2'], yticklabels=['D0', 'D1', 'D2'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.ensemble import RandomForestRegressor

# Setup
SEQ_LENGTH = 12
categories = ['D0', 'D1', 'D2', 'D3', 'D4']
results = {}

# Convert date column
final_historical_df['date'] = pd.to_datetime(final_historical_df['date'])

# Feature matrix (constant across models)
features = final_historical_df.drop(columns=['date', 'D0', 'D1', 'D2', 'D3', 'D4'])
feature_scaler = MinMaxScaler()
X_scaled = feature_scaler.fit_transform(features)

# Sequence creation function
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])
    return np.array(X_seq), np.array(y_seq)

# Loop over drought categories
for cat in categories:
    print(f"\n🚀 Training model for {cat}")

    # Prepare target
    y = final_historical_df[[cat]].values
    target_scaler = MinMaxScaler()
    y_scaled = target_scaler.fit_transform(y)

    # Create sequences
    X_seq, y_seq = create_sequences(X_scaled, y_scaled, SEQ_LENGTH)

    # Train-test split
    split_idx = int(0.8 * len(X_seq))
    X_train, X_test = X_seq[:split_idx], X_seq[split_idx:]
    y_train, y_test = y_seq[:split_idx], y_seq[split_idx:]

    # Build LSTM model
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(SEQ_LENGTH, X_train.shape[2])),
        Dropout(0.3),
        LSTM(32),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])

    # Train
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=100,
        batch_size=32,
        callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
        verbose=0
    )

    # Predict
    y_pred_scaled = model.predict(X_test)
    y_pred = target_scaler.inverse_transform(y_pred_scaled)
    y_test_orig = target_scaler.inverse_transform(y_test)

    # Metrics
    mae = mean_absolute_error(y_test_orig, y_pred)
    mse = mean_squared_error(y_test_orig, y_pred)
    r2 = r2_score(y_test_orig, y_pred)

    print(f"{cat} MAE: {mae:.4f}, MSE: {mse:.4f}, R²: {r2:.4f}")

    # Save results
    results[cat] = {
        'true': y_test_orig.flatten(),
        'pred': y_pred.flatten(),
        'mae': mae,
        'mse': mse,
        'r2': r2
    }

    # Plot
    plt.figure(figsize=(10, 4))
    plt.plot(y_test_orig, label='True')
    plt.plot(y_pred, label='Predicted')
    plt.title(f'{cat} Prediction')
    plt.xlabel('Time Step')
    plt.ylabel('Percentage Area')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Optional: Random Forest comparison
    X_train_flat = X_train.reshape(X_train.shape[0], -1)
    X_test_flat = X_test.reshape(X_test.shape[0], -1)

    rf = RandomForestRegressor(n_estimators=100)
    rf.fit(X_train_flat, y_train.ravel())
    rf_pred = target_scaler.inverse_transform(rf.predict(X_test_flat).reshape(-1, 1))

    rf_mae = mean_absolute_error(y_test_orig, rf_pred)
    rf_r2 = r2_score(y_test_orig, rf_pred)

    print(f"🌳 Random Forest {cat} MAE: {rf_mae:.4f}, R²: {rf_r2:.4f}")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import pandas as pd

# Replace this with your actual dataframe
# final_historical_df = pd.read_csv('your_data.csv')

# Drop any rows with missing values
final_historical_df = final_historical_df.dropna()

# Features to use
X_raw = final_historical_df.drop(columns=['date', 'D0', 'D1', 'D2', 'D3', 'D4'])

# Store results
metrics = {}

# Train a separate model for each drought level
for target_col in ['D0', 'D1', 'D2', 'D3', 'D4']:
    print(f"\nTraining model for {target_col}...")

    # Target column
    y_raw = final_historical_df[target_col].values.reshape(-1, 1)

    # Scale features and target
    X_scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()

    X = X_scaler.fit_transform(X_raw)
    y = y_scaler.fit_transform(y_raw)

    # Time-aware split (or shuffle=True for random)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, shuffle=False)

    # Define small neural network
    model = Sequential([
        Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])

    # Train the model
    model.fit(X_train, y_train, epochs=100, batch_size=32,
              validation_split=0.2, verbose=0)

    # Predict and inverse transform
    y_pred_scaled = model.predict(X_test)
    y_pred = y_scaler.inverse_transform(y_pred_scaled)
    y_true = y_scaler.inverse_transform(y_test)

    # Evaluate
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    metrics[target_col] = {'mae': mae, 'r2': r2}

# Print results
print("\n=== Evaluation Results ===")
for cat, m in metrics.items():
    print(f"{cat} - MAE: {m['mae']:.4f}, R²: {m['r2']:.4f}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, BatchNormalization

# Load and clean your dataframe
# final_historical_df = pd.read_csv('your_data.csv')
final_historical_df = final_historical_df.dropna()

# Features (drop date + target drought levels)
X_raw = final_historical_df.drop(columns=['date', 'D0', 'D1', 'D2', 'D3', 'D4'])

# Targets (D0-D4)
y_raw = final_historical_df[['D0', 'D1', 'D2', 'D3', 'D4']].values

# Scale features and targets
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

X = X_scaler.fit_transform(X_raw)
y = y_scaler.fit_transform(y_raw)

# Train-test split (time-aware)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# Define a more complex neural network architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(5)  # 5 outputs: D0, D1, D2, D3, D4
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32,
                    validation_split=0.2, verbose=0)

# Predict and inverse scale
y_pred_scaled = model.predict(X_test)
y_pred = y_scaler.inverse_transform(y_pred_scaled)
y_true = y_scaler.inverse_transform(y_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

# Display overall MAE and R²
print(f"Overall MAE: {mae:.4f}")
print(f"Overall R²: {r2:.4f}")

# Evaluate per drought level
for i, level in enumerate(['D0', 'D1', 'D2', 'D3', 'D4']):
    mae_level = mean_absolute_error(y_true[:, i], y_pred[:, i])
    r2_level = r2_score(y_true[:, i], y_pred[:, i])
    print(f"{level} - MAE: {mae_level:.4f}, R²: {r2_level:.4f}")

    # Plot for each drought level
    plt.figure(figsize=(10, 4))
    plt.plot(y_true[:, i], label='True')
    plt.plot(y_pred[:, i], label='Predicted')
    plt.title(f"{level} - True vs Predicted")
    plt.xlabel("Time Step")
    plt.ylabel("Percentage Area")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# # Define multi-output neural network
# model = Sequential([
#     Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
#     Dense(32, activation='relu'),
#     Dense(16, activation='relu'),
#     Dense(5)  # 5 outputs: D0, D1, D2, D3, D4
# ])

# model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# # Train the model
# history = model.fit(X_train, y_train, epochs=100, batch_size=32,
#                     validation_split=0.2, verbose=0)

# # Predict and inverse scale
# y_pred_scaled = model.predict(X_test)
# y_pred = y_scaler.inverse_transform(y_pred_scaled)
# y_true = y_scaler.inverse_transform(y_test)

# # Calculate evaluation metrics
# mae = mean_absolute_error(y_true, y_pred)
# r2 = r2_score(y_true, y_pred)

# # Display overall MAE and R²
# print(f"Overall MAE: {mae:.4f}")
# print(f"Overall R²: {r2:.4f}")

# # Evaluate per drought level
# for i, level in enumerate(['D0', 'D1', 'D2', 'D3', 'D4']):
#     mae_level = mean_absolute_error(y_true[:, i], y_pred[:, i])
#     r2_level = r2_score(y_true[:, i], y_pred[:, i])
#     print(f"{level} - MAE: {mae_level:.4f}, R²: {r2_level:.4f}")

#     # Plot for each drought level
#     plt.figure(figsize=(10, 4))
#     plt.plot(y_true[:, i], label='True')
#     plt.plot(y_pred[:, i], label='Predicted')
#     plt.title(f"{level} - True vs Predicted")
#     plt.xlabel("Time Step")
#     plt.ylabel("Percentage Area")
#     plt.legend()
#     plt.grid(True)
#     plt.tight_layout()
#     plt.show()

Failed D4 model -- bad predictions

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.losses import Huber
import matplotlib.pyplot as plt

# Assuming final_historical_df is your dataframe
final_historical_df['date'] = pd.to_datetime(final_historical_df['date'])

# Create Lag Features (1-step lag for D0, D1, D2, D3, D4)
for col in ['D0', 'D1', 'D2', 'D3', 'D4']:
    for lag in range(1, 4):  # Lag 1, 2, 3
        final_historical_df[f'{col}_lag{lag}'] = final_historical_df[col].shift(lag)

# Create Rolling Mean and Std Features (using a window of 7 days, for example)
for col in ['D0', 'D1', 'D2', 'D3', 'D4']:
    final_historical_df[f'{col}_rolling_mean'] = final_historical_df[col].rolling(window=7).mean()
    final_historical_df[f'{col}_rolling_std'] = final_historical_df[col].rolling(window=7).std()

# Drop NaN values generated by shifting and rolling operations
final_historical_df = final_historical_df.dropna()

# Prepare Features and Target specifically for D4
features = final_historical_df.drop(columns=['date', 'D0', 'D1', 'D2', 'D3', 'D4'])
target = final_historical_df[['D4']]

# Normalize Features and Target
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

X_scaled = feature_scaler.fit_transform(features)
y_scaled = target_scaler.fit_transform(target)

# Create Sequences for LSTM
SEQ_LENGTH = 12  # 12 months of data for time series forecasting

X_seq, y_seq = [], []
for i in range(len(X_scaled) - SEQ_LENGTH):
    X_seq.append(X_scaled[i:i+SEQ_LENGTH])  # Sequence of features
    y_seq.append(y_scaled[i+SEQ_LENGTH])  # Next time step prediction for D4

X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

# Train/Test Split
split_idx = int(0.8 * len(X_seq))  # 80% train, 20% test
X_train, X_test = X_seq[:split_idx], X_seq[split_idx:]
y_train, y_test = y_seq[:split_idx], y_seq[split_idx:]

# Build and Train the LSTM Model for D4
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(SEQ_LENGTH, X_train.shape[2])),
    Dropout(0.3),
    LSTM(32),
    Dense(1)  # Output for D4 (single output regression task)
])

# Compile the model with Huber loss and Adam optimizer
model.compile(optimizer='adam', loss=Huber(), metrics=['mae'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=200,
    batch_size=32,
    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)]
)

# Evaluate the Model
y_pred_scaled = model.predict(X_test)

# Inverse transform predictions and true labels to original scale
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_orig = target_scaler.inverse_transform(y_test)

# Calculate MAE and R² for D4
from sklearn.metrics import mean_absolute_error, r2_score

mae = mean_absolute_error(y_test_orig, y_pred)
r2 = r2_score(y_test_orig, y_pred)
print(f"D4 - MAE: {mae:.4f}, R²: {r2:.4f}")

# Plot predictions vs true values for D4
plt.figure(figsize=(10, 4))
plt.plot(y_test_orig, label='True')
plt.plot(y_pred, label='Predicted')
plt.title("D4 - True vs Predicted")
plt.xlabel("Time Step")
plt.ylabel("Percentage Area")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

Just D4

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.losses import Huber
import matplotlib.pyplot as plt

# Assuming final_historical_df is your dataframe
final_historical_df['date'] = pd.to_datetime(final_historical_df['date'])

# Create Lag Features (1-step lag for D0, D1, D2, D3, D4)
for col in ['D0', 'D1', 'D2', 'D3', 'D4']:
    for lag in range(1, 4):  # Lag 1, 2, 3
        final_historical_df[f'{col}_lag{lag}'] = final_historical_df[col].shift(lag)

# Create Rolling Mean and Std Features (using a window of 7 days, for example)
for col in ['D0', 'D1', 'D2', 'D3', 'D4']:
    final_historical_df[f'{col}_rolling_mean'] = final_historical_df[col].rolling(window=7).mean()
    final_historical_df[f'{col}_rolling_std'] = final_historical_df[col].rolling(window=7).std()

# Drop NaN values generated by shifting and rolling operations
final_historical_df = final_historical_df.dropna()

# Prepare Features and Target specifically for D4
features = final_historical_df.drop(columns=['date', 'D0', 'D1', 'D2', 'D3', 'D4'])
target = final_historical_df[['D4']]

# Normalize Features and Target
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

X_scaled = feature_scaler.fit_transform(features)
y_scaled = target_scaler.fit_transform(target)

# Create Sequences for LSTM
SEQ_LENGTH = 12  # 12 months of data for time series forecasting

X_seq, y_seq = [], []
for i in range(len(X_scaled) - SEQ_LENGTH):
    X_seq.append(X_scaled[i:i+SEQ_LENGTH])  # Sequence of features
    y_seq.append(y_scaled[i+SEQ_LENGTH])  # Next time step prediction for D4

X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

# Train/Test Split
split_idx = int(0.8 * len(X_seq))  # 80% train, 20% test
X_train, X_test = X_seq[:split_idx], X_seq[split_idx:]
y_train, y_test = y_seq[:split_idx], y_seq[split_idx:]

# Build and Train the LSTM Model for D4
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(SEQ_LENGTH, X_train.shape[2])),
    Dropout(0.3),
    LSTM(32),
    Dense(1)  # Output for D4 (single output regression task)
])

# Compile the model with Huber loss and Adam optimizer
model.compile(optimizer='adam', loss=Huber(), metrics=['mae'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=200,
    batch_size=32,
    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)]
)

# Evaluate the Model
y_pred_scaled = model.predict(X_test)

# Inverse transform predictions and true labels to original scale
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_orig = target_scaler.inverse_transform(y_test)

# Calculate MAE and R² for D4
from sklearn.metrics import mean_absolute_error, r2_score

mae = mean_absolute_error(y_test_orig, y_pred)
r2 = r2_score(y_test_orig, y_pred)
print(f"D4 - MAE: {mae:.4f}, R²: {r2:.4f}")

# Plot predictions vs true values for D4
plt.figure(figsize=(10, 4))
plt.plot(y_test_orig, label='True')
plt.plot(y_pred, label='Predicted')
plt.title("D4 - True vs Predicted")
plt.xlabel("Time Step")
plt.ylabel("Percentage Area")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### Random Forest and XGBoost Regressors

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt

# Step 1: Feature Engineering
df = final_historical_df.copy()
df['date'] = pd.to_datetime(df['date'])

# Create lag features
for col in ['D0', 'D1', 'D2', 'D3', 'D4']:
    for lag in range(1, 4):
        df[f'{col}_lag{lag}'] = df[col].shift(lag)

# Create rolling stats (7-day window)
for col in ['D0', 'D1', 'D2', 'D3', 'D4']:
    df[f'{col}_roll_mean'] = df[col].rolling(window=7).mean()
    df[f'{col}_roll_std'] = df[col].rolling(window=7).std()

# Drop NaNs caused by lag and rolling
df = df.dropna().reset_index(drop=True)

# Step 2: Features and Targets
X = df.drop(columns=['date', 'D0', 'D1', 'D2', 'D3', 'D4'])
y = df[['D0', 'D1', 'D2', 'D3', 'D4']]

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Time-aware split (80% train, 20% test)
split_idx = int(0.8 * len(X))
X_train, X_test = X_scaled[:split_idx], X_scaled[split_idx:]
y_train, y_test = y[:split_idx].values, y[split_idx:].values

# Step 3: Fit and Evaluate Models
results = {}

for model_name, model_class in {
    'RandomForest': RandomForestRegressor,
    'XGBoost': XGBRegressor
}.items():
    
    print(f"\n====== {model_name} Results ======")
    model_results = {}
    
    for i, target in enumerate(['D0', 'D1', 'D2', 'D3', 'D4']):
        model = model_class(n_estimators=100, random_state=42)
        model.fit(X_train, y_train[:, i])
        y_pred = model.predict(X_test)
        
        mae = mean_absolute_error(y_test[:, i], y_pred)
        r2 = r2_score(y_test[:, i], y_pred)
        model_results[target] = {'MAE': mae, 'R2': r2}
        
        print(f"{target} - MAE: {mae:.4f}, R²: {r2:.4f}")
        
        # Plot predictions
        plt.figure(figsize=(10, 4))
        plt.plot(y_test[:, i], label='True')
        plt.plot(y_pred, label='Predicted')
        plt.title(f"{model_name} - {target}")
        plt.xlabel("Time Step")
        plt.ylabel("Percentage Area")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()
    
    results[model_name] = model_results
